In [ ]:
from azureml.core import Workspace

from azureml.core.authentication import ServicePrincipalAuthentication

svc_pr_password = '3:sft5wyl3W.:avsw@FhR3X.?7DWQn0F'

svc_pr = ServicePrincipalAuthentication(
    tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47",
    service_principal_id="xxxxxxxxxxxxxxxxxx",
    service_principal_password=svc_pr_password)


ws = Workspace.from_config(auth=svc_pr)

print("Found workspace {} at location {}".format(ws.name, ws.location))


In [ ]:
svc_pr.get_authentication_header()

In [ ]:
from azureml.pipeline.core import PublishedPipeline
import requests

pipeline_endpoint = 'https://australiaeast.api.azureml.ms/pipelines/v1.0/subscriptions/907c8efc-c2c8-4c49-a4e1-aeb880e10c88/resourceGroups/aml/providers/Microsoft.MachineLearningServices/workspaces/magaml/PipelineRuns/PipelineSubmit/6b8e8468-6eee-442f-8eb7-c20347b64be0'

response = requests.post(pipeline_endpoint,
                         headers=svc_pr.get_authentication_header(),
                         json={'ExperimentName': 'My_Pipeline',
                               'ParameterAssignments': {
                                   'pipeline_inpart': '2020/07/20',
                                    'pipeline_kv_customimg': '123',
                                    'pipeline_kv_readapi': '342'}})

In [ ]:
import json
from collections import namedtuple 
resp_str = response.content
resp = json.loads(response.content)
# https://www.geeksforgeeks.org/convert-json-data-into-a-custom-python-object/
resp_o = json.loads(resp_str, object_hook = lambda d: namedtuple('X', d.keys())(*d.values()))

In [ ]:
print(resp_o.RunUrl)
print(resp_o.Status.StatusCode)
print('pipeline id: ', resp_o.PipelineId)
print('run id: ', resp_o.Id)

In [ ]:
resp.keys()

In [ ]:
resp

# authenticate using requests
https://docs.microsoft.com/en-us/azure/machine-learning/how-to-manage-rest

In [ ]:
tenant_id = '72f988bf-86f1-41af-91ab-2d7cd011db47'
client_id = 'xxxxxxxxxxxxxxxxxxxxxxx'
client_secret = svc_pr_password
auth_request_url = f'https://login.microsoftonline.com/{tenant_id}/oauth2/token'
auth_request_data = f'grant_type=client_credentials&resource=https%3A%2F%2Fmanagement.azure.com%2F&client_id={client_id}&client_secret={client_secret}'

In [ ]:
print(auth_request_url)
print(auth_request_data)
print(pipeline_endpoint)

In [ ]:

auth_response = requests.post(auth_request_url,data=auth_request_data)
auth_resp_dict = json.loads(auth_response.content)


In [ ]:
auth_resp_dict['access_token']

In [ ]:
rest_response = requests.post(pipeline_endpoint,
                         headers={'Authorization': 'Bearer ' + auth_resp_dict['access_token']},
                         json={'ExperimentName': 'My_Pipeline_pure_rest',
                               'ParameterAssignments': {
                                   'pipeline_inpart': '2020/07/20',
                                    'pipeline_kv_customimg': '123',
                                    'pipeline_kv_readapi': '342'}})

In [ ]:
json.loads(rest_response.content)

In [ ]:
from azureml.pipeline.core import PipelineRun
from azureml.core import Experiment

In [ ]:
pipeline_run = PipelineRun(experiment=Experiment(workspace=ws, name='My_Pipeline_pure_rest'),run_id='1353cce4-acf6-4d00-a440-3bcdea617c03')

In [ ]:
ctx = pipeline_run.get_context()

In [ ]:
ctx

In [ ]:
pipeline_run.wait_for_completion()

# create dataset using REST 

```
https://<your-workspace-region>.experiments.azureml.net/dataset/v1.0/subscriptions/<your-subscription-id>/resourceGroups/<your-resource-group-name>/providers/Microsoft.MachineLearningServices/workspaces/<your-workspace-name>/saveddatasets/from-data-path
```

In [ ]:
workspaceregion = 'australiaeast'
subscriptionid = '907c8efc-c2c8-4c49-a4e1-aeb880e10c88'
resourcegroupname = 'aml'
workspacename = 'magaml'
datastorename = 'godzilla'
relativepath = 'images/2020/07/28'
header = {'Authorization': 'Bearer ' + auth_resp_dict['access_token']}
dataset_body = {
    'DatastoreName': f'{datastorename}',
    'RelativePath': f'{relativepath}'}

create_dataset_rest = f'https://{workspaceregion}.experiments.azureml.net/dataset/v1.0/subscriptions/{subscriptionid}/resourceGroups/{resourcegroupname}/providers/Microsoft.MachineLearningServices/workspaces/{workspacename}/saveddatasets/from-data-path'

In [ ]:
dataset_rest_response = requests.post(create_dataset_rest,
                         headers=header,
                         json=dataset_body)

In [ ]:
dataset_resp_dict = json.loads(dataset_rest_response.content)

In [ ]:
json.loads(dataset_resp_dict['dataflowJson'])

In [ ]:
dataset_resp_o = json.loads(dataset_resp_dict['dataflowJson'], object_hook = lambda d: namedtuple('X', d.keys())(*d.values()))

In [ ]:
saved_dataset_id = dataset_resp_o.meta.savedDatasetId